In [1]:
import cv2
import numpy as np
from detection2 import Yaw


# name = "IMG_3603"
# name = "IMG_3468"

# name = "IMG_4295"
# name = "IMG_4296"
# name = "IMG_4297"
# name = "IMG_4299"
# name = "IMG_4300"
# name = "IMG_4301"
# name = "IMG_4302"

name = "IMG_4384"
# name = "IMG_4385"
# name = "IMG_4386"
# name = "IMG_4387"


FRAME_PER_SECOND = 4


cap = cv2.VideoCapture('sh/mov/'+ name +'.MOV')

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps    = cap.get(cv2.CAP_PROP_FPS)

shoter_time = int(fps // FRAME_PER_SECOND)

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps    = cap.get(cv2.CAP_PROP_FPS)
frames = []
angles = []
yaw_obj = Yaw()


c= 1
while True:
    ret, frame = cap.read()
    if not ret:
        break

    c+=1

    yaww = yaw_obj.predict_yaw(frame)
    angles.append(yaww)
    frames.append(frame)

    if (c% 100 ==0):
        print(c , yaww)

    
cap.release()


Using cache found in C:\Users\Farhad/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-7-10 Python-3.9.12 torch-1.11.0+cu113 CUDA:0 (NVIDIA GeForce GTX 1050, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


Instructions for updating:
dim is deprecated, use axis instead


From c:\Users\Farhad\anaconda3\envs\gputest\lib\site-packages\tensorflow\python\util\dispatch.py:1082: calling l2_normalize (from tensorflow.python.ops.nn_impl) with dim is deprecated and will be removed in a future version.
Instructions for updating:
dim is deprecated, use axis instead


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                             

In [2]:
video_array = np.array(frames)

# real-time

In [3]:
def find_nearest(lst, num):
    if num is  None:
        return None

    nearest_num = lst[0]
    nearest_idx = 0
    min_diff = 99999999

    if lst[0] is not None:
        min_diff = abs(num - lst[0])
    
    
    for i in range(1, len(lst) ):
        if lst[i] is  None:
            continue
        diff = abs(num - lst[i])
        if diff <= min_diff:
            nearest_num = lst[i]
            nearest_idx = i
            min_diff = diff
            
    return (nearest_num, nearest_idx)


def rotate_180(angle):
    return (angle + 180) % 360

In [4]:

important_angles = [0 , 60 , 90  , 110 , 190 , 240 , 270 , 305 , 350]
angle_names = ["left" , "front-left" , "front" , "front-right" , "right" , "back-right" , "back" , "back-left" , "left"]

def find_main_angle(angle , expectation_idx = 0): # expectation_idx is previous angle_idx

    # old mode
    # important_angles = [0 , 30 , 90  , 150 , 180 , 200 , 270 , 340 , 360]
    # angle_names = ["front" , "front-left" , "left" , "back-left" , "back" , "back-right" , "right" , "front-right" , "front"]
    
    # correct mode


    # temp mode
    # important_angles = [ 270 , 200 , 180 , 150 , 90 , 30 , 0  , 340 , 290]
    # angle_names = [ "right" , "back-right" , "back" , "back-left" , "left" , "front-left" , "front" , "front-right" , "right"]

    nearest = find_nearest(important_angles , angle)
    if nearest is None:
        return "---" , expectation_idx # nothing detected
    
    ang , idx = nearest
    if 280 > abs(ang - important_angles[expectation_idx]) > 100: # dont mistake on 180 degree
        angle2 = rotate_180(angle)
        ang , idx = find_nearest(important_angles , angle2)
    if idx != expectation_idx + 1: # and  idx != expectation_idx - 1: # dont turn back
        idx = expectation_idx
    return angle_names[idx] , idx
    
    


In [5]:
import os
import random


font = cv2.FONT_HERSHEY_SIMPLEX
x, y = int(video_array[0].shape[1] // 2.5) , video_array[0].shape[0] // 2
color = (150, 35, 200)
font_scale = 2
thickness = 10

r = random.randint(0,12000)

In [6]:
import cv2
import os
import copy
# Set the directory containing the images
li = copy.deepcopy(video_array[:,::3,::3,:])
print("fffffffffffffffffff")
x, y = int(li[0].shape[1] // 2.3) , li[0].shape[0] // 2
# Create a window to display the video
cv2.namedWindow('Video', cv2.WINDOW_NORMAL)

# Loop through each image and display it in the window
c= 0 
expectation_idx = 0
yawObj = Yaw()

yaws = []
main_angles = []

detected_video = []

for ii in range(li.shape[0]):
    if c % shoter_time == 0 :
        yaww = yawObj.predict_yaw(video_array[ii])
        ang , idx = find_main_angle(yaww , expectation_idx = expectation_idx)
        yaws.append(yaww)
        main_angles.append(ang)
        expectation_idx = idx
    cv2.putText(li[ii], str(ang), (x,y), font, font_scale, color, thickness)
    cv2.imshow('Video', li[ii])
    detected_video.append(li[ii])
    c+=1
    # Wait for 25 milliseconds between each frame (40 frames per second)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the window and close all windows
cv2.destroyAllWindows()

fffffffffffffffffff


Using cache found in C:\Users\Farhad/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-7-10 Python-3.9.12 torch-1.11.0+cu113 CUDA:0 (NVIDIA GeForce GTX 1050, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                           

In [7]:
detected_video = np.array(detected_video)
size = detected_video.shape[2] , detected_video.shape[1]

fps = 30


out = cv2.VideoWriter('sh/mov/RECO/___new8_CORR_'+ name+"_"+ str(r) +'reco.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, size)

for f in detected_video:
    out.write(f)
out.release() 

# light frame-selection

In [8]:
video_array.shape 

(1826, 1080, 1920, 3)

In [9]:
shots = copy.deepcopy(video_array[::shoter_time,:,:])

shots.shape[0] , len(main_angles) , len(yaws)


(261, 261, 261)

In [11]:
np.where(np.array(main_angles) == 'left')[0]

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260], dtype=int64)

In [ ]:
important_angles = [0 , 60 , 90  , 110 , 190 , 240 , 270 , 305 , 350]
angle_names = ["left" , "front-left" , "front" , "front-right" , "right" , "back-right" , "back" , "back-left" , "left"]




def light_frame_selection(shots , main_angles , yaws , num_of_frame = 100):
    # delete non-car
    selected_frames = []
    selected_yaws = []
    selected_main_angles = []

    for i in range(len(main_angles) - 1 , -1 , -1):
        if main_angles[i] == "---":
            del main_angles[i]
            del yaws[i]
            shots = np.delete(shots, i)


    for i in range(0 , len(important_angles)):
        if i != 0 and i != len(important_angles) - 1 : 
            angle_range = (important_angles[i + 1] -  important_angles[i - 1]) / 2 
        if i == 0 :
            angle_range = (important_angles[i + 1] +  important_angles[0]) / 2
        if i == len(important_angles) - 1 :
            angle_range = 360 - ((important_angles[i] +  important_angles[i - 1]) / 2)

        num_frame_in_one_mainAngle = angle_range * num_of_frame / 360
        # print(angle_names[i] , main_angles[i])

        indices = np.where(np.array(main_angles) == angle_names[i])[0]
        # print(indices)
        if i == 0 :
            for ii in range(len(indices) - 1, -1, -1):
                if indices[ii] > len(main_angles) / 2 :
                    indices = np.delete(indices, ii)

        if i == len(important_angles) - 1 :
            for ii in range(len(indices) - 1, -1, -1):
                if indices[ii] <= len(main_angles) / 2 :
                    indices = np.delete(indices, ii)

        shot1 = [shots[i] for i in indices]
        main_angles1 = [main_angles[i] for i in indices]
        yaws1 = [yaws[i] for i in indices]

        jump = int(len(yaws1) / num_frame_in_one_mainAngle)

        # print(i , len(yaws1) , num_frame_in_one_mainAngle , indices)


        selected_frames += shot1[::jump]
        selected_yaws += yaws1[::jump]
        selected_main_angles += main_angles1[::jump]

    return selected_frames , selected_yaws , selected_main_angles


            


In [ ]:
a , b , c = light_frame_selection(shots , main_angles , yaws , num_of_frame = 100)

In [ ]:

# Path
# Create the directory
# 'GeeksForGeeks' in
# '/home / User / Documents'
os.mkdir("sh/mov/"+name+"__"+ str(r))

In [ ]:
# List of images
images = a # Replace with your list of NumPy arrays

# Write images to disk
for i, image in enumerate(images):
    filename = f"sh/mov/"+ name+"__"+ str(r) +f"/image_{i}.png"
    cv2.imwrite(filename, image)

# other 

In [14]:
def select_frame_from_period(frames ,angles , angle_range):
    print(angle_range ," -------> ", angles)
    a , b = angle_range
    angle_avg = (a+b) / 2
    angle , idx = find_nearest(angles , angle_avg)
    if angle is  None:
        return None
    if 280 > abs(angle - angle_avg) > 100:
        angle_avg = rotate_180(angle_avg)
        angle , idx = find_nearest(angles , angle_avg)
        angle = rotate_180(angle)
    return frames[idx] , angle





In [15]:
def frame_selection(angles , frames , num_frames = 30):
    selected_frames = []
    selected_angles = []
    angle_period = 360 // num_frames
    frame_period = frames.shape[0] // num_frames
    for i in range(num_frames):
        # print(i)
        half_period = angle_period // 2
        a = i * angle_period - half_period
        b = (i+1) * angle_period - half_period - 1
        c = i * frame_period
        d = (i+1) * frame_period -1
        
        frame_angle = select_frame_from_period( frames[c:d,:,:,:],angles[c:d] , angle_range = (a,b))

        if frame_angle is None:
            continue

        frame , angle = frame_angle
        
        
        selected_frames.append(frame)
        selected_angles.append(angle)

    return selected_frames , selected_angles




        

In [16]:
sel_frames , sel_angles = frame_selection(angles , video_array , 25)

(-7, 6)  ------->  [70.41964977575316, 70.80191340216771, 69.6086814888306, 70.79589856467913, 70.96620939253046, 71.3893491544224, 73.40993754568171, 70.94194684437588, 72.05456200611366]
(7, 20)  ------->  [69.82954597989988, 69.90563175313844, 67.55386956872417, 66.73105047246736, 67.3072800147213, 68.78929328319799, 67.13951087192368, 67.04455157171265, 66.46103848387821]
(21, 34)  ------->  [64.17906719735782, 63.85578825193272, 62.647661825124146, 63.26673651903121, 62.02795469816249, 62.185063832844776, 60.88424909117538, 60.4689615950374, 56.977209214429536]
(35, 48)  ------->  [50.854865577138185, 48.20217966444568, 41.90132704837691, 42.43058664357437, 39.832158065501574, 39.97361816398279, 38.167669397912164, 38.10663651321306, 37.116472172925626]
(49, 62)  ------->  [35.24595760554445, 34.80306919837966, 32.30001570843234, 31.21733300766124, 16.059573668318194, 15.931651009612438, 16.985945726495174, 18.306556444530884, 19.611071816032336]
(63, 76)  ------->  [22.2646080875

In [17]:
sel_angles

[69.6086814888306,
 66.46103848387821,
 56.977209214429536,
 41.90132704837691,
 35.24595760554445,
 22.26460808758054,
 13.095830584316616,
 2.0827794695152875,
 157.57220720723785,
 136.8841697855538,
 134.40568961747948,
 123.26896998612943,
 121.37153934115236,
 245.2606409441492,
 241.72967381881276,
 226.7123634273222,
 223.26959424610942,
 207.34149581788265,
 204.50961470559918,
 181.09095145230003,
 180.7000074527594,
 335.20019554074486,
 320.9734288842045,
 316.8597718158446,
 304.21464748251515]

In [18]:
sel_frames = np.array(sel_frames)
size = sel_frames.shape[2] , sel_frames.shape[1]

fps = 2


out = cv2.VideoWriter('sh/mov/'+ name+"__"+ str(r) +'/reco_.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, size)

for f in sel_frames:
    out.write(f)
out.release() 


In [19]:

# List of images
images = sel_frames # Replace with your list of NumPy arrays

# Write images to disk
for i, image in enumerate(images):
    filename = f"sh/mov/"+ name+"__"+ str(r) +f"/image_{i}.png"
    cv2.imwrite(filename, image)